### Imports and dependencies

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
sns.set_style("ticks")

### Get dataframes by meterids and obtain list of meterids.

In [9]:
df_all = pd.read_csv('data/dataport-export_gas_oct2015-mar2016.csv')
df_all = df_all.set_index(pd.to_datetime(df_all['localminute']))
df_all = df_all.drop(columns='localminute')
groups = df_all.groupby('dataid')
keys = groups.groups.keys()  # keys: an iterable of dataids or meter ids
keys_list = list(keys)
print(keys_list)

[35, 44, 77, 94, 114, 187, 222, 252, 370, 483, 484, 661, 739, 744, 871, 1042, 1086, 1103, 1185, 1283, 1403, 1415, 1507, 1556, 1589, 1619, 1697, 1714, 1718, 1790, 1791, 1792, 1800, 1801, 2018, 2034, 2072, 2094, 2129, 2233, 2335, 2378, 2449, 2461, 2470, 2575, 2638, 2645, 2755, 2814, 2818, 2945, 2946, 2965, 2980, 3036, 3039, 3134, 3310, 3367, 3527, 3544, 3577, 3635, 3723, 3778, 3849, 3893, 3918, 4029, 4031, 4193, 4228, 4296, 4352, 4356, 4373, 4421, 4447, 4514, 4671, 4732, 4767, 4874, 4998, 5129, 5131, 5193, 5275, 5317, 5395, 5403, 5439, 5484, 5545, 5636, 5658, 5785, 5810, 5814, 5892, 5972, 6101, 6412, 6505, 6578, 6673, 6685, 6830, 6836, 6863, 6910, 7016, 7017, 7030, 7117, 7287, 7429, 7460, 7566, 7674, 7682, 7739, 7741, 7794, 7900, 7919, 7965, 7989, 8059, 8084, 8086, 8155, 8156, 8244, 8386, 8467, 8703, 8829, 8890, 8967, 9052, 9121, 9134, 9160, 9278, 9295, 9474, 9600, 9620, 9631, 9639, 9729, 9766, 9849, 9956, 9982]


In [13]:
key = 35
df_i = groups.get_group(key).drop(columns='dataid')

In [14]:
def zoom(df, start_date, end_date):
    # returns a df whose rows are within a particular time period. 
    # pre-condition: df is indexed by datetime.
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    mask = (df.index >= start_date) & (df.index <= end_date)

    new_df = df.iloc[mask]
    return new_df

In [12]:
# example for zoom
df_i_1mth = zoom(df_i, '2016-01-01', '2016-02-01')

TypeError: Cannot compare type 'Timestamp' with type 'int'

### Cleaning steps.

In [ ]:
# step 1 (optional part)
def remove_mal_data(df):
    # this method finds the start datetime and end datetime of the malfunctioning period and returns a new df without data
    # from that period. i.e. the entire chunk of data from the faulty period is dropped.
    # NOTE that a new column 'marginal_change' is added to the df.
    # df: pandas dataframe. contains data from one meterid. contains a column 'meter_value'.
    
    # from visualising the data, we can arbitrarily define a spike as a marginal difference of > 2000 cubic metres
    # we have defined malfunction as a marginal change > 2000
    # and a marginal change < -2000.
    threshold = 2000
    df['marginal_change'] = df['meter_value'].diff()
    
    flagged_dates = df[df.marginal_change > threshold].index
    flagged_dates = flagged_dates.append(df[df.marginal_change < -1*threshold].index)
    
    if len(flagged_dates) > 0:
        start = flagged_dates[0]
        end = flagged_dates[-1]    
        mask = (df.index >= start) & (df.index <= end)
        return df.loc[~mask]
    else:
        return df

In [ ]:
# step 2. exclude meters with low sampling frequency

# list of meters visually inspected and identified to be malfunctioning 
# based on spikes or irregularities seen on the plot.
to_remove = [1185, 1556, 2335, 2449, 3134, 3544, 4447, 4514,\
             5129, 5403, 6836, 7030, 7117, 8156, 9134, 9639, 9982]

low_samplefreq = [1874, 2645, 4671, 5545, 6101, 9620]

to_remove.extend(low_samplefreq) 

display(to_remove, len(to_remove))

# final meters to use for analysis
to_keep = [elem for elem in keys_list if elem not in to_remove]

In [ ]:
# step 3. Change cumulative to marginal change.

# we already have the 'marginal_change' column. 
# NOTE: the 'meter_value' or cumulative reading column is NO LONGER VALID after dropping any row.
# the only exception is for the very first row which is the first cumulative reading.
# if we are going to need cumulative readings again, we must construct them from this first cumulative reading and all
# the new (remaining/cleaned) marginal changes.

In [ ]:
# step 4. remove marginal decrease.
def remove_negative_marginal(df):
    # remove data points where the marginal change (from prev value) is negative.
    return df.loc[df['marginal_change'] >= 0]

# same notes apply as in step 3.

In [ ]:
# step 5. Get rate of increase from marginal changes (per second)

In [ ]:
# step 6. Mark anomalies from rate of increase with threshold = 0.166/s
# step 7. Remove these anomalies
# step 8. Resample hourly, finding the new hourly marginal reading. Construct cumulative readings as necessary.
# step 9. visualise cleaned readings. confirm that results are as intended
# step 10. save combined data into a new csv file.